# Part 2 - Warehouse Analytics 

In [1]:
import pandas as pd
import numpy as np

products_df = pd.read_csv("../data/warehouse_products.csv")
products_df.head()

,date,product_id,warehouse_id,demand,inbound_units_weekly,actual_outbound_weekly,inventory_level_weekly,unmet_demand_weekly,stockout_flag_weekly,inbound_units_jit,actual_outbound_jit,inventory_level_jit,unmet_demand_jit,stockout_flag_jit
0,2024-01-01,P001,WH1,29,0,29,221,0,False,0,29,221,0,False
1,2024-01-01,P002,WH1,23,0,23,227,0,False,0,23,227,0,False
2,2024-01-01,P003,WH1,34,0,34,216,0,False,0,34,216,0,False
3,2024-01-01,P004,WH1,21,0,21,229,0,False,0,21,229,0,False
4,2024-01-01,P005,WH1,28,0,28,222,0,False,0,28,222,0,False


In [2]:
selected_columns = ["date",
                    "warehouse_id",
                    "inbound_units_weekly",
                    "actual_outbound_weekly",
                    "inventory_level_weekly",
                    "unmet_demand_weekly",
                    "inbound_units_jit",
                    "actual_outbound_jit",
                    "inventory_level_jit",
                    "unmet_demand_jit"
                   ]

df = products_df[selected_columns]

In [3]:
df.head()

,date,warehouse_id,inbound_units_weekly,actual_outbound_weekly,inventory_level_weekly,unmet_demand_weekly,inbound_units_jit,actual_outbound_jit,inventory_level_jit,unmet_demand_jit
0,2024-01-01,WH1,0,29,221,0,0,29,221,0
1,2024-01-01,WH1,0,23,227,0,0,23,227,0
2,2024-01-01,WH1,0,34,216,0,0,34,216,0
3,2024-01-01,WH1,0,21,229,0,0,21,229,0
4,2024-01-01,WH1,0,28,222,0,0,28,222,0


In [4]:
group_df = df.groupby(["date", "warehouse_id"])
group_df = group_df.agg({
    "inbound_units_weekly": "sum",
    "actual_outbound_weekly": "sum",
    "inventory_level_weekly": "sum",
    "unmet_demand_weekly": "sum",
    "inbound_units_jit": "sum",
    "actual_outbound_jit": "sum",
    "inventory_level_jit": "sum",
    "unmet_demand_jit": "sum"})
group_df = group_df.reset_index()

In [5]:
group_df = group_df.rename(columns={
    "actual_outbound_weekly": "orders_fulfilled_weekly",
    "unmet_demand_weekly": "missed_sales_weekly",
    "actual_outbound_jit": "orders_fulfilled_jit",
    "unmet_demand_jit": "missed_sales_jit"
})

In [6]:
van_capacity = 100
group_df["outbound_shipments_weekly"] = group_df["orders_fulfilled_weekly"] / van_capacity
group_df["outbound_shipments_jit"] = group_df["orders_fulfilled_jit"] / van_capacity

In [7]:
group_df["outbound_shipments_weekly"] = group_df["outbound_shipments_weekly"].apply(np.ceil).astype("int")
group_df["outbound_shipments_jit"] = group_df["outbound_shipments_jit"].apply(np.ceil).astype("int")

In [8]:
warehouse_capacity = 2500

group_df["utilization_weekly"] = group_df["inventory_level_weekly"] / warehouse_capacity
group_df["utilization_jit"] = group_df["inventory_level_jit"] / warehouse_capacity

In [9]:
large_truck_capacity = 1000

group_df["inbound_shipments_weekly"] = group_df["inbound_units_weekly"] / large_truck_capacity
group_df["inbound_shipments_weekly"] = group_df["inbound_shipments_weekly"].apply(np.ceil).astype("int")
group_df["truck_utilization_weekly"] = group_df["inbound_units_weekly"] / (group_df["inbound_shipments_weekly"] * large_truck_capacity)

In [10]:
small_truck_capacity = 500

group_df["inbound_shipments_jit"] = group_df["inbound_units_jit"] / small_truck_capacity
group_df["inbound_shipments_jit"] = group_df["inbound_shipments_jit"].apply(np.ceil).astype("int")
group_df["truck_utilization_jit"] = group_df["inbound_units_jit"] / (group_df["inbound_shipments_jit"] * small_truck_capacity)

In [11]:
group_df

,date,warehouse_id,inbound_units_weekly,orders_fulfilled_weekly,inventory_level_weekly,missed_sales_weekly,inbound_units_jit,orders_fulfilled_jit,inventory_level_jit,missed_sales_jit,outbound_shipments_weekly,outbound_shipments_jit,utilization_weekly,utilization_jit,inbound_shipments_weekly,truck_utilization_weekly,inbound_shipments_jit,truck_utilization_jit
0,2024-01-01,WH1,0,223,1627,0,0,223,1627,0,3,3,0.6508,0.6508,0,NaN,0,NaN
1,2024-01-02,WH1,0,201,1426,0,0,201,1426,0,3,3,0.5704,0.5704,0,NaN,0,NaN
2,2024-01-03,WH1,0,222,1204,0,0,222,1204,0,3,3,0.4816,0.4816,0,NaN,0,NaN
3,2024-01-04,WH1,0,216,988,0,0,216,988,0,3,3,0.3952,0.3952,0,NaN,0,NaN
4,2024-01-05,WH1,0,258,730,0,0,258,730,0,3,3,0.2920,0.2920,0,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,2024-03-26,WH1,0,102,1801,0,0,102,1071,0,2,2,0.7204,0.4284,0,NaN,0,NaN
86,2024-03-27,WH1,0,118,1683,0,330,116,1285,2,2,2,0.6732,0.5140,0,NaN,1,0.66
87,2024-03-28,WH1,0,114,1569,0,0,114,1171,0,2,2,0.6276,0.4684,0,NaN,0,NaN
88,2024-03-29,WH1,0,145,1424,0,200,145,1226,0,2,2,0.5696,0.4904,0,NaN,1,0.40


def determine_staff_count(row):
    outbound_orders = row["orders_fulfilled"]
    inbound_shipments = row["inbound_shipments"]
    if outbound_orders < 100:
        return 2
    elif outbound_orders <= 150:
        return 3
    elif outbound_orders <= 200:
        return 4
    else:
        return 5

group_df["staff_count"] = group_df.apply(determine_staff_count, axis=1)

group_df["staff_count"] = group_df.apply(determine_staff_count, axis=1)

In [12]:
group_df.to_csv("../data/warehouse_daily.csv", index=False)